# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [115]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [273]:
import pandas as pd
import numpy as np
import glob
from datetime import date,time,datetime
import time
import xlrd
# Write your answer here

input the data

In [274]:
path = r'Data\ebola\guinea_data'
allFiles = glob.glob(path + "/*.csv")
frame1 = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None,header=0)
    list_.append(df)
frame1 = pd.concat(list_)

path = r'Data\ebola\liberia_data'
allFiles = glob.glob(path + "/*.csv")
frame2 = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None,header=0)
    list_.append(df)
frame2 = pd.concat(list_)

path = r'Data\ebola\sl_data'
allFiles = glob.glob(path + "/*.csv")
frame3 = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None,header=0)
    list_.append(df)
frame3 = pd.concat(list_)

rename the "Description" and "Variable" colunmn in order to make the same columns for the three countries

In [275]:
#rename the "Description" colunmn in order to make the same columns for the three countries
frame1 = frame1.rename(index=str, columns={"Description": "Variable"})
frame3 = frame3.rename(index=str, columns={"variable": "Variable","date": "Date"})

add the tap of country

In [ ]:
frame1['Country'] = 'guinea'
frame2['Country'] = 'liberia'
frame3['Country'] = 'sl'

in order to combine the datas of three countries, we make them the same 'Variable'

In [277]:
frame1.rename(index=str, columns={"Totals": "National"},inplace=True)
frame2.replace('Total confirmed cases','Total cases of confirmed',inplace=True)
frame2.replace('Total probable cases','Total cases of probables',inplace=True)
frame2.replace('Total suspected cases','Total cases of suspects',inplace=True)
frame2.replace('Total death/s in confirmed cases','Total deaths of confirmed',inplace=True)
frame2.replace('Total death/s in probable cases','Total deaths of probables',inplace=True)
frame2.replace('Total death/s in suspected cases','Total deaths of suspects',inplace=True)

frame3.replace('cum_confirmed','Total cases of confirmed',inplace=True)
frame3.replace('cum_probable','Total cases of probables',inplace=True)
frame3.replace('cum_suspected','Total cases of suspects',inplace=True)
frame3.replace('death_confirmed','Total deaths of confirmed',inplace=True)
frame3.replace('death_probable','Total deaths of probables',inplace=True)
frame3.replace('death_suspected','Total deaths of suspects',inplace=True)

since we only need to calculate data for each countriy, so we chose the data we want.

In [278]:
frame1 = frame1[['Date','Country','Variable','National']]
frame2 = frame2[['Date','Country','Variable','National']]
frame3 = frame3[['Date','Country','Variable','National']]

merge data of three countries

In [279]:
DF = pd.merge(frame1,frame2,how='outer')
df = pd.merge(DF,frame3,how='outer')
df['Date'] = pd.to_datetime(df.Date)

since 10th month only has one day's value so we should take out the value

In [302]:
a = df.loc[(df['Date']=='2014-10-01') & (df['Country']=='guinea') & (df['Variable']=='Total new cases registered so far')]
b = df.loc[(df['Date']=='2014-10-01') & (df['Country']=='guinea') & (df['Variable']=='New deaths registered')]
c = int(a.National.values[0])
d = int(b.National.values[0])

get the data about 'Total cases' and 'Total deaths'.
In my opinion, the other data such as 'New cases of suspects' are not necessary data for us
to calculate daily average per month of new cases and deaths, since data with 'Total' can truely reflect the situation while the 'New' can
only reflect specific days' situation rather than per month.

In [285]:
sub1 = df[[name.startswith('Total cases of') for name in df.Variable]]
sub2 = df[[name.startswith('Total deaths of') for name in df.Variable]]

 merge the dataframe into a single dataframe

In [286]:
Data1 = pd.merge(sub1,sub2,how='outer')

In [287]:
Data= Data1.set_index(['Country','Variable','Date']).sort_index()

In [288]:
Data = Data.dropna()

In [289]:
Data.loc['sl','Total cases of confirmed','2014-09-06']=1276
Data.loc['sl','Total cases of confirmed','2014-09-17']=1585
Data.loc['sl','Total cases of confirmed','2014-09-18']=1618
Data.loc['sl','Total cases of confirmed','2014-09-20']=1696
Data.loc['sl','Total cases of confirmed','2014-09-27']=2000

In [290]:
Str = ['guinea','liberia','sl']
String = ['Total cases of confirmed','Total cases of probables','Total cases of suspects','Total deaths of confirmed','Total deaths of probables','Total deaths of suspects']

choose the data we wanted
since values of 'National' are strings, we should transfer them into int or float
we use the data of one month's last day subtract the data of month's first day

In [291]:
List=[]
Ave = np.zeros([3,6])
for i in range(0,3):
    for j in range(0,6):
        sub = Data.loc[Str[i],String[j]] 
        sub['National'] = pd.to_numeric(sub.National
        sub = sub.reset_index()
        A = sub.groupby(sub['Date'].map(lambda x: x.month)).last()
        B = sub.groupby(sub['Date'].map(lambda x: x.month)).first()
        diff = A - B 
        ave = np.zeros([len(diff),1])
        for t in range (0,len(diff)):
            if diff.Date.loc[diff.index[t]].days==0:
                ave[t] = np.nan
            else:
                ave[t] = diff.National.loc[diff.index[t]]/diff.Date.loc[diff.index[t]].days
        AA = pd.DataFrame()
        AA= pd.DataFrame(ave)
        AA['Month'] = diff.index 
        AA['Country']=Str[i]
        AA['Variable'] = String[j]
        List.append(AA)
C = pd.concat(List)

In [307]:
C.columns=['National','Month','Country','Variable']
C.fillna(0,inplace=True)
C.head(10)

,National,Month,Country,Variable
0,7.851852,8,guinea,Total cases of confirmed
1,13.214286,9,guinea,Total cases of confirmed
2,0.000000,10,guinea,Total cases of confirmed
0,0.629630,8,guinea,Total cases of probables
1,0.892857,9,guinea,Total cases of probables
2,0.000000,10,guinea,Total cases of probables
0,0.925926,8,guinea,Total cases of suspects
1,-0.571429,9,guinea,Total cases of suspects
2,0.000000,10,guinea,Total cases of suspects
0,4.037037,8,guinea,Total deaths of confirmed


Conbine the Total cases/deaths of confirmed,probabales and suspects as Total cases/deaths

In [308]:
Str = ['guinea','liberia','sl']
St = ['Total cases','Total deaths']
listt=[]
for i in range(0,3):
    for j in range(0,2):
        C1 = C.loc[C.Country==Str[i]]
        C11 = C1.loc[C1.Variable.str.startswith(St[j])]
        C111=C11.National.groupby(C11.Month).sum()
        C111 = pd.DataFrame(C111)
        C111['Country']=Str[i]
        C111['Variable']=St[j]
        listt.append(C111)
data = pd.concat(listt)

In [309]:
data= data.reset_index()

change the data for octorber since it only has one day's data

In [318]:
data.National[2]=c
data.National[5]=d

d:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
d:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [319]:
data.set_index(['Country','Variable','Month'])

National
Country Variable     Month           
guinea  Total cases  8       9.407407
                     9      13.535714
                     10     34.000000
        Total deaths 8       4.666667
                     9      14.636905
                     10     15.000000
liberia Total cases  6       5.230769
                     7       8.280000
                     8      42.730769
                     9      71.137931
                     10      8.566667
                     11     40.321429
                     12     22.000000
        Total deaths 6       2.538462
                     7       3.360000
                     8      24.153846
                     9      36.344828
                     10     37.266667
                     11      0.000000
sl      Total cases  8      20.526316
                     9      41.931034
                     10     71.033333
                     11     86.535714
                     12     80.083333
        Total deaths 8       6.947368
                     9       5.310345
                     10     28.500000
                     11     15.000000
                     12     31.800000

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [609]:
#input data and set the colunms
path = r'Data\microbiome'
allFiles = glob.glob(path + "/*.xls")
dfs = pd.DataFrame()
list_= []
i = 1
for file_ in allFiles[1:10]:
    df= pd.read_excel(file_, 'Sheet 1', index_col=None, header=None)
    String = 'MID'+str(i)
    df['BARCODE'] =String
    i = i + 1
    dfs = pd.concat([dfs,df],axis=0)
dfs.columns = ['Taxon','Count','BARCODE']

In [610]:
X_file = pd.ExcelFile(allFiles[0])
Df=X_file.parse('Sheet1')

In [605]:
#add the metadata information from the 10th spreadsheet as columns in the combined DataFrame
Data = pd.merge(dfs,Df,on='BARCODE',right_index = True)

In [611]:
#all the NaN values have been replaced by the tag unknown
Data.fillna('unknown')

,Taxon,Count,BARCODE,GROUP,SAMPLE
0,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",7,MID1,EXTRACTION CONTROL,unknown
1,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2,MID1,EXTRACTION CONTROL,unknown
2,"Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",3,MID1,EXTRACTION CONTROL,unknown
3,"Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",3,MID1,EXTRACTION CONTROL,unknown
4,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",7,MID1,EXTRACTION CONTROL,unknown


In [608]:
Data.set_index(['Taxon','GROUP','SAMPLE']).sort_index()

Count  \
Taxon                                              GROUP              SAMPLE          
Archaea "Crenarchaeota" Thermoprotei Acidilobal... Control 1          tissue      1   
                                                   Control 2          tissue      5   
                                                   NEC 1              tissue      2   
Archaea "Crenarchaeota" Thermoprotei Acidilobal... Control 1          stool       1   
                                                                      tissue     15   
                                                   Control 2          stool       1   
                                                                      tissue     26   
                                                   NEC 1              tissue     14   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          stool       8   
                                                                      tissue     14   
                                                   Control 2          stool      16   
                                                                      tissue     28   
                                                   EXTRACTION CONTROL NaN         7   
                                                   NEC 1              stool       7   
                                                                      tissue     23   
                                                   NEC 2              tissue      2   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 2          tissue      1   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          stool       2   
                                                                      tissue      4   
                                                   Control 2          stool       2   
                                                                      tissue      5   
                                                   NEC 1              stool       1   
                                                                      tissue      1   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          tissue      1   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          stool       1   
                                                                      tissue      1   
                                                   Control 2          tissue      2   
                                                   NEC 1              tissue      2   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... NEC 1              tissue      1   
Archaea "Crenarchaeota" Thermoprotei Desulfuroc... Control 1          stool       1   
...                                                                             ...   
Bacteria "Verrucomicrobia" Subdivision3   Subdi... Control 1          stool       4   
Bacteria "Verrucomicrobia" Verrucomicrobiae Ver... Control 1          stool       9   
                                                                      tissue      2   
                                                   NEC 1              stool       1   
Bacteria "Verrucomicrobia" Verrucomicrobiae Ver... Control 1          stool      39   
                                                                      tissue      1   
                                                   NEC 1              stool       1   
Bacteria Cyanobacteria Cyanobacteria  Chloropla... Control 1          stool       1   
                                                                      tissue      1   
                                                   Control 2          tissue      2   
                                                   EXTRACTION CONTROL NaN         2   
                                                   NEC 2              stool       1   
Bacteria Cyanobacteria Cyanobacteria  Chloropla... Control 1          stool       2   
                                                                      tissue

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.